# Setting Pyspark

In [31]:
import findspark

from configs.config import DATA_SOURCE_DIR
from pyspark.sql import SparkSession
from train.transforms.splitting import stratified_splitting
from train.transforms.outliers import log_transformation
from train.transforms.categorical_data import *
from train.transforms.correlation import *

ImportError: cannot import name 'log_transformation' from 'train.transforms.outliers' (/Users/haiduong/Documents/mlops-loan-approval-classification/train/transforms/outliers.py)

In [4]:
findspark.init()
findspark.find()

spark= SparkSession \
       .builder \
       .appName("Feature engineering 1") \
       .getOrCreate()

24/12/24 15:30:41 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [5]:
df = spark.read.csv(str(DATA_SOURCE_DIR),header=True,escape="\"")
df_pandas = df.toPandas()

# Stratified Splitting

In [ ]:
train_df, test_df, val_df = stratified_splitting(
    df_pandas, train_size=0.8, test_size=0.1, val_size=0.1, random_state=42
)

Confirm the proportion of class

In [25]:
def test_stratified_sampling(source_df, 
                             train_df,
                             test_df,
                             val_df
                            ):
    source_df_class_proportion = source_df['loan_status'].value_counts()[0] / source_df['loan_status'].value_counts()[1]
    train_df_class_proportion = train_df['loan_status'].value_counts()[0] / train_df['loan_status'].value_counts()[1]
    val_df_class_proportion = val_df['loan_status'].value_counts()[0] / val_df['loan_status'].value_counts()[1]
    test_df_class_proportion = test_df['loan_status'].value_counts()[0] / test_df['loan_status'].value_counts()[1]

    if source_df_class_proportion == train_df_class_proportion == val_df_class_proportion == test_df_class_proportion:
        print("The proportion of class in each set are the same")

In [26]:
test_stratified_sampling(
    df_pandas, train_df, test_df, val_df
)

The proportion of class in each set are the same


/var/folders/13/jltvwky966g5t57g1b1zpn5h0000gn/T/ipykernel_14377/1785067673.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  source_df_class_proportion = source_df['loan_status'].value_counts()[0] / source_df['loan_status'].value_counts()[1]
/var/folders/13/jltvwky966g5t57g1b1zpn5h0000gn/T/ipykernel_14377/1785067673.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  source_df_class_proportion = source_df['loan_status'].value_counts()[0] / source_df['loan_status'].value_counts()[1]
/var/folders/13/jltvwky966g5t57g1b1zpn5h0000gn/T/ipykernel_14377/1785067673.py:7: FutureWarning: Series.__getitem__ treating keys as p

Convert back to pyspark dataframe

In [27]:
train_spark_df = spark.createDataFrame(train_df)
test_spark_df = spark.createDataFrame(test_df)
val_spark_df = spark.createDataFrame(val_df)